那 LCEL 有什么优势呢？
LCEL 从底层设计的目标就是支持 从原型到生产 完整流程不需要修改任何代码，也就是我们在写的任何原型代码不需要太多的改变就能支持生产级别的各种特性（比如并行、steaming 等），具体来说会有这些优势：

- 并行，只要是整个 chain 中有可以并行的步骤就会自动的并行，来减少使用时的延迟。
- 自动的重试和 fallback。大部分 chain 的组成部分都有自动的重试（比如因为网络原因的失败）和回退机制，来解决很多请求的出错问题。 而不需要我们去写代码 cover 这些问题。
- 对 chain 中间结果的访问，在旧的写法中很难访问中间的结果，而 LCEL 中可以方便的通过访问中间结果来进行调试和记录。
- LCEL 会自动支持 LangSimith 进行可视化和记录。这是 langchain 官方推出的记录工具，可以记录一条 chian 运行过程中的大部分信息，来方便调试 LLM 找到是哪些中间环节的导致了最终结果较差。这部分我们会在后续的章节中涉及到。


# invoke


In [1]:
import { ChatOpenAI } from "@langchain/openai"
import { HumanMessage } from "@langchain/core/messages"

const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

await model.invoke([
  new HumanMessage("Tell me a joke"),
])


AIMessage {
  "id": "chatcmpl-B7P4dGluhVkUMPGllFQrEgaxFyNXV",
  "content": "Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 11,
      "completionTokens": 18,
      "totalTokens": 29
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "completion_tokens": 18,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens": 11,
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      },
      "total_tokens": 29
    },
    "system_fingerprint": "fp_b705f0c291"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 18,
    "input_tokens": 11,
    "total_tokens": 29,
    "input_token_details": {
    

为了方便展示，我们会加入一个简单的 `StringOutputParser` 来处理输出，你可以简单的理解为将 OpenAI 返回的复杂对象提取出最核心的字符串，更详细的 `OutputParser` 相关介绍会在后续章节中展开。组成一个最基础的 Chain 来演示， `Runnable` 中各个调用方式


In [2]:
import { StringOutputParser } from "@langchain/core/output_parsers"

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

const outputPrase = new StringOutputParser()

const simpleChain = chatModel.pipe(outputPrase)

await simpleChain.invoke([
  new HumanMessage("Tell me a joke"),
])


"Why don't scientists trust atoms? \n\nBecause they make up everything!"

通过 StringOutputParser，我们获取了 AIMessage 中 content 的值。在 LCEL 中，使用 `.pipe()` 方法来组装多个 `Runnable` 对象形成完整的 Chain，可以看到我们是用对单个模块同样的 `invoke` 方法去调用整个 chain。 因为无论是单个模块还是由模块组装而成的多个 chain 都是 `Runnable`。


# Patch


然后我们尝试对这个基础的 Chain 进行批量调用，用起来也非常简单


In [4]:
await simpleChain.batch([
  [new HumanMessage("Tell me a joke")],
  [new HumanMessage("Hi, Who are you?")],
])


[
  "Why don't scientists trust atoms?\n\nBecause they make up everything!",
  "Hello! I’m an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and engage in conversation on a variety of topics. How can I help you today?"
]

# Stream


因为 LLM 的很多调用都是一段一段的返回的，如果等到完整地内容再返回给用户，就会让用户等待比较久，影响用户的体验。而 LCEL 开箱就支持 steaming，即所谓的流式传输，我们依旧使用我们定义的基础 Chain，就可以直接获得 streaming 的能力


In [6]:
const stream = await simpleChain.stream([
  new HumanMessage("Tell me a joke"),
])

for await (const chunk of stream) {
  console.log(chunk)
}



Why
 did
 the
 scare
crow
 win
 an
 award
?
 


Because
 he
 was
 outstanding
 in
 his
 field
!




streamLog 的使用较少，他会在每次返回 chunk 的时候，返回完整的对象，我们不深入介绍，感兴趣的可以运行下述代码观察其每个 chunk 的返回值，并根据自己需要去使用。


In [7]:
const stream = await simpleChain.streamLog([
  new HumanMessage("Tell me a joke"),
])

for await (const chunk of stream) {
  console.log(chunk)
}


RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "1083ced0-fd36-4590-83d4-84360ba2d9ea",
        name: "RunnableSequence",
        type: "chain",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI",
      value: {
        id: "bbad8453-0d7a-4c0f-a119-0a838d129bfb",
        name: "ChatOpenAI",
        type: "llm",
        tags: [ "seq:step:1" ],
        metadata: {
          ls_provider: "openai",
          ls_model_name: "gpt-4o-mini",
          ls_model_type: "chat",
          ls_temperature: undefined,
          ls_max_tokens: undefined,
          ls_stop: undefined
        },
        start_time: "2025-03-04T16:20:47.178Z",
        streamed_output: [],
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      pat

# fallback

`withFallbacks` 是任何 runnable 都有的一个函数，可以给当前 runnable 对象添加 fallback 然后生成一个带 fallback 的 `RunnableWithFallbacks` 对象，这适合我们将自己的 fallback 逻辑增加到 LCEL 中。

例如，我们创建一个一定会失败的 llm ：

> 注意这里要多等一会（1-2min）


In [1]:
import { ChatOpenAI } from "@langchain/openai"

const fakeLLM = new ChatOpenAI({
  openAIApiKey: "123",
  maxRetries: 0,
})

await fakeLLM.invoke("你好")


Error: Connection error.

而正确的呢？下述代码也是用了很久返回了正确的内容


In [3]:
const realLLM = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

const llmWithFallback = fakeLLM.withFallbacks({
  fallbacks: [realLLM],
})

await llmWithFallback.invoke("你好")


AIMessage {
  "id": "chatcmpl-B7PZj7DvVH1C3igNV1SchHGZIOckJ",
  "content": "你好！有什么我可以帮助你的吗？",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 8,
      "completionTokens": 9,
      "totalTokens": 17
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "completion_tokens": 9,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens": 8,
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      },
      "total_tokens": 17
    },
    "system_fingerprint": "fp_b705f0c291"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 9,
    "input_tokens": 8,
    "total_tokens": 17,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
    "output_token_details": 

因为无论是 llm model 或者其他的模块，还是整个 chain 都是 runnable 对象，所以我们可以给整个 LCEL 流程中的任意环节去增加 fallback，来避免一个环节出问题卡住剩下环境的运行。

当然，我们也可以给整个 chain 增加 fallback，例如一个复杂但输出高质量的结果的 chain 可以设置一个非常简单的 chain 作为 fallback，可以在极端环境下保证至少有输出。


## 小结

That's All!  
这就是 langchain.js 基础，如果你在 LCEL 之前学习过 langchain，你会发现 LCEL 极大的降低了 langchain 的使用难度，并且为使用 chain 提供了开箱即用的生产级能力支持。其最大的魅力就是进一步强化了模块化，可以方便的复用各种 chain 来组合成更复杂的 chain。

所以我认为，在当前时间点可以抛弃之前的旧写法，全面拥抱 LCEL。 在掌握了基础用法后，我们就可以探索更多 langchain 的应用了。至于更多 LCEL 的高级用法，我们会随着实战逐步引入和讲解。
